A notebook to run the entire pipeline using the ground-truth noise classifier during the graph-contruction phase

In [1]:
from functools import partial
import os

import torch

from gnn_tracking.training.tc import TCModule
from gnn_tracking.training.ml import MLModule
from gnn_tracking.models.graph_construction import MLGraphConstructionFromChkpt, GraphConstructionFCNN, NoiseClassifierModel
from gnn_tracking.models.track_condensation_networks import GraphTCNForMLGCPipeline
from gnn_tracking.graph_construction.k_scanner import GraphConstructionKNNScanner
from gnn_tracking.metrics.losses.metric_learning import GraphConstructionHingeEmbeddingLoss
from gnn_tracking.postprocessing.dbscanscanner import DBSCANHyperParamScanner
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint, LearningRateMonitor
from gnn_tracking.utils.loading import TrackingDataModule
from gnn_tracking.training.callbacks import PrintValidationMetrics, ExpandWandbConfig
from gnn_tracking.utils.versioning import assert_version_geq

from torch_geometric.data import Data
from torch import nn

assert_version_geq("23.12.0")

# Configure the data

In [2]:
data_path_pixel = "/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v8/"
data_paths_pixel = os.listdir(data_path_pixel)
data_paths_pixel.sort()
data_paths_pixel = list(map(lambda x: data_path_pixel + x, data_paths_pixel))

In [3]:
data_path_fd = "/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v10/"
data_paths_fd = os.listdir(data_path_fd)
data_paths_fd.sort()
data_paths_fd = list(map(lambda x: data_path_fd + x, data_paths_fd))

In [4]:
dm = TrackingDataModule(
    train=dict(
        dirs=data_paths_pixel[1:-1],
        sample_size=900
    ),
    val=dict(
        dirs=[data_paths_pixel[-1]],
        start=0,
        stop=4,
    ),
    identifier="point_clouds_v8",
)
dm.setup(stage='fit')

[05:33:37] INFO: DataLoader will load 7743 graphs (out of 7743 available).
[05:33:37] DEBUG: First graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v8/part_1/data21000_s0.pt, last graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v8/part_8/data28999_s0.pt
[05:33:37] INFO: DataLoader will load 4 graphs (out of 1000 available).
[05:33:37] DEBUG: First graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v8/part_9/data29000_s0.pt, last graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v8/part_9/data29003_s0.pt


In [5]:
dm_fd = TrackingDataModule(
    train=dict(
        dirs=data_paths_fd[1:-1],
        sample_size=900
    ),
    val=dict(
        dirs=[data_paths_fd[-1]],
        start=0,
        stop=4,
    ),
    identifier="point_clouds_v10",
)
dm_fd.setup(stage='fit')

[09:14:17] INFO: DataLoader will load 7743 graphs (out of 7743 available).
[09:14:17] DEBUG: First graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v10/part_1/data21000_s0.pt, last graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v10/part_8/data28999_s0.pt
[09:14:17] INFO: DataLoader will load 4 graphs (out of 1000 available).
[09:14:17] DEBUG: First graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v10/part_9/data29000_s0.pt, last graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v10/part_9/data29003_s0.pt


In [9]:
tmp = list(map(lambda x: x.size()[0], dm_pixel.datasets['train']))

In [52]:
tmp_ = list(map(lambda x: x.size()[0], dm_fd.datasets['train']))

In [53]:
np.savez('./strip_mask.npz', tmp_)

In [13]:
d1 = dm_pixel.datasets['train'][0]
d2 = dm_fd.datasets['train'][0]

In [39]:
import numpy as np

mask = np.ones(d1.num_nodes)
mask = np.hstack([mask, np.zeros(d2.num_nodes - d1.num_nodes)])
mask = torch.tensor(mask, dtype=bool)

pixel_hits = d2.subgraph(mask)
strip_hits = d2.subgraph(~mask)

In [16]:
torch.all(d1.layer == d2.layer[:d1.num_nodes])

tensor(True)

In [18]:
the_truth = []
for particle_id in d1.particle_id:
    the_truth.append(particle_id in d2.particle_id)

# GC-phase

In [5]:
noise_model = NoiseClassifierModel()
model = GraphConstructionFCNN(in_dim=14, out_dim=8, depth=6, hidden_dim=256)

In [6]:
lmodel = MLModule(
    model=model,
    loss_fct=GraphConstructionHingeEmbeddingLoss(
        lw_repulsive=0.06,
        max_num_neighbors=256,
    ),
    optimizer=partial(torch.optim.Adam, lr=7*1e-4),
    preproc=noise_model,
    gc_scanner=GraphConstructionKNNScanner(ks=list(range(1, 11)))
)

[05:34:38] WARNING: Can't save hyperparameters from object of type <class 'gnn_tracking.models.graph_construction.NoiseClassifierModel'>. Make sure to inherit from HyperparametersMixin.


In [9]:
from pytorch_lightning.loggers import TensorBoardLogger, WandbLogger
from wandb_osh.lightning_hooks import TriggerWandbSyncLightningCallback
from gnn_tracking.utils.nomenclature import random_trial_name

name = random_trial_name()

wandb_logger_gc = WandbLogger(
    project="aryaman-gnn-experiments",
    group="noise-classification-gc",
    offline=True,
    version=name,
    tags=["noise-classification-gc-with-scanner"],
)

tb_logger_gc = TensorBoardLogger(".", version=name)

──────────────────────────── chubby-stoic-goldfish ─────────────────────────────

In [12]:
trainer_gc = Trainer(
    max_epochs=100,
    accelerator="gpu",
    log_every_n_steps=1,
    callbacks=[
        TriggerWandbSyncLightningCallback(),
        PrintValidationMetrics(),
    ],
    logger=[
        wandb_logger_gc,
        tb_logger_gc,
    ],
)
trainer_gc.fit(model=lmodel, datamodule=dm)

05:37:14 INFO: This is wandb-osh v1.2.0 using communication directory /home/aj2239/.wandb_osh_command_dir
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
[05:37:14] INFO: DataLoader will load 7743 graphs (out of 7743 available).
[05:37:14] DEBUG: First graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v8/part_1/data21000_s0.pt, last graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v8/part_8/data28999_s0.pt
[05:37:14] INFO: DataLoader will load 4 graphs (out of 1000 available).
[05:37:14] DEBUG: First graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v8/part_9/data29000_s0.pt, last graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v8/part_9/data29003_s0.pt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type                                | Params
--------------------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/scratch/gpfs/aj2239/micromamba/envs/gnn/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.
/scratch/gpfs/aj2239/micromamba/envs/gnn/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

                      Validation epoch=98                       
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Metric                           ┃        Value ┃      Error ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ attractive                       │      0.44674 │    0.01063 │
│ attractive_train                 │      0.44992 │        nan │
│ attractive_weighted              │      0.44674 │    0.01063 │
│ attractive_weighted_train        │      0.44992 │        nan │
│ efficiency_at_max_frac_segment50 │      0.74091 │        nan │
│ efficiency_at_segment50_80       │      0.23928 │        nan │
│ efficiency_at_segment50_85       │      0.27543 │        nan │
│ efficiency_at_segment50_88       │      0.30852 │        nan │
│ efficiency_at_segment50_90       │      0.33951 │        nan │
│ efficiency_at_segment50_93       │      0.41459 │        nan │
│ efficiency_at_segment50_95       │      0.50551 │        nan │
│ efficiency_at_segment50

Validation: |          | 0/? [00:00<?, ?it/s]

08:09:02 WARNING: Syncing not active or too slow: Command /home/aj2239/.wandb_osh_command_dir/9b5780.command file still exists
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.


`Trainer.fit` stopped: `max_epochs=100` reached.


                      Validation epoch=99                       
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Metric                           ┃        Value ┃      Error ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ attractive                       │      0.46054 │    0.01154 │
│ attractive_train                 │      0.46081 │        nan │
│ attractive_weighted              │      0.46054 │    0.01154 │
│ attractive_weighted_train        │      0.46081 │        nan │
│ efficiency_at_max_frac_segment50 │      0.72801 │        nan │
│ efficiency_at_segment50_80       │      0.24375 │        nan │
│ efficiency_at_segment50_85       │      0.28275 │        nan │
│ efficiency_at_segment50_88       │      0.31877 │        nan │
│ efficiency_at_segment50_90       │      0.35218 │        nan │
│ efficiency_at_segment50_93       │      0.43502 │        nan │
│ efficiency_at_segment50_95       │      0.53442 │        nan │
│ efficiency_at_segment50

In [7]:
trainer_gc = Trainer(
    max_epochs=100,
    accelerator="gpu",
    log_every_n_steps=1,
    callbacks=[
        TriggerWandbSyncLightningCallback(),
        PrintValidationMetrics(),
    ],
    logger=[
        wandb_logger_gc,
        tb_logger_gc,
    ],
)
trainer_gc.fit(model=lmodel, datamodule=dm)

08:16:00 INFO: This is wandb-osh v1.2.0 using communication directory /home/aj2239/.wandb_osh_command_dir
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100 80GB PCIe') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING `resume` will be ignored since W&B syncing is set to `offline`. Starting a new run with run id miniature-divergent-prawn.


[08:16:02] INFO: DataLoader will load 7743 graphs (out of 7743 available).
[08:16:02] DEBUG: First graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v8/part_1/data21000_s0.pt, last graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v8/part_8/data28999_s0.pt
[08:16:02] INFO: DataLoader will load 4 graphs (out of 1000 available).
[08:16:02] DEBUG: First graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v8/part_9/data29000_s0.pt, last graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v8/part_9/data29003_s0.pt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type                                | Params
-----------------------------------------------------------------
0 | model    | GraphConstructionFCNN               | 333 K 
1 | preproc  | NoiseClassifierModel                | 0     
2 | loss_fct | GraphConstructionHingeEmbeddingLoss | 0     
--------------------------------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/scratch/gpfs/aj2239/micromamba/envs/gnn/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.
/scratch/gpfs/aj2239/micromamba/envs/gnn/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

                  Validation epoch=72                   
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Metric                    ┃       Value ┃      Error ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ attractive                │     0.39732 │    0.00615 │
│ attractive_train          │     0.38386 │        nan │
│ attractive_weighted       │     0.39732 │    0.00615 │
│ attractive_weighted_train │     0.38386 │        nan │
│ n_edges_att               │ 35077.50000 │ 1273.15723 │
│ n_edges_att_train         │ 22211.00000 │        nan │
│ n_edges_rep               │ 92185.50000 │ 7146.31445 │
│ n_edges_rep_train         │ 42992.00000 │        nan │
│ n_hits_oi                 │  9646.25000 │  405.12701 │
│ n_hits_oi_train           │  6091.00000 │        nan │
│ repulsive                 │     2.79627 │    0.12222 │
│ repulsive_train           │     2.02669 │        nan │
│ repulsive_weighted        │     0.16778 │    0.00733 │
│ repulsive_weighted_train  │  

Validation: |          | 0/? [00:00<?, ?it/s]

                  Validation epoch=73                   
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Metric                    ┃       Value ┃      Error ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ attractive                │     0.40428 │    0.00787 │
│ attractive_train          │     0.40001 │        nan │
│ attractive_weighted       │     0.40428 │    0.00787 │
│ attractive_weighted_train │     0.40001 │        nan │
│ n_edges_att               │ 35077.50000 │ 1273.15723 │
│ n_edges_att_train         │ 31440.00000 │        nan │
│ n_edges_rep               │ 91551.00000 │ 7137.53760 │
│ n_edges_rep_train         │ 75000.00000 │        nan │
│ n_hits_oi                 │  9646.25000 │  405.12701 │
│ n_hits_oi_train           │  8469.00000 │        nan │
│ repulsive                 │     2.77289 │    0.12026 │
│ repulsive_train           │     2.63062 │        nan │
│ repulsive_weighted        │     0.16637 │    0.00722 │
│ repulsive_weighted_train  │  

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The Jupyter serve

# Rest of the OC pipeline

In [4]:
model = GraphTCNForMLGCPipeline(
    node_indim=22,
    edge_indim=44,
    h_dim=192,
    e_dim=192,
    hidden_dim=192,
    h_outdim=24,
    L_hc=5,
    alpha_latent=0.5,
    n_embedding_coords=8
)

/scratch/gpfs/aj2239/micromamba/envs/gnn/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:198: Attribute 'hc_in' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['hc_in'])`.


In [5]:
preproc = MLGraphConstructionFromChkpt(
    ml_chkpt_path="/home/aj2239/aryaman-gnn-tracking-experiments/notebooks/aryaman-gnn-experiments/miniature-divergent-prawn/checkpoints/epoch=99-step=774300.ckpt",
    max_num_neighbors=10,
    max_radius=1,
    use_embedding_features=True,
    build_edge_features=True
)

[13:51:59] DEBUG: Getting class MLModule from module gnn_tracking.training.ml
[13:51:59] DEBUG: Loading checkpoint /home/aj2239/aryaman-gnn-tracking-experiments/notebooks/aryaman-gnn-experiments/miniature-divergent-prawn/checkpoints/epoch=99-step=774300.ckpt
[13:52:00] DEBUG: Getting class GraphConstructionFCNN from module gnn_tracking.models.graph_construction
[13:52:00] DEBUG: Getting class NoiseClassifierModel from module gnn_tracking.models.graph_construction
[13:52:00] DEBUG: Getting class GraphConstructionHingeEmbeddingLoss from module gnn_tracking.metrics.losses.metric_learning
[13:52:00] DEBUG: Checkpoint loaded. Model ready to go.


In [6]:
loss_fct = GraphConstructionHingeEmbeddingLoss(
    lw_repulsive=0.05,
    pt_thld=0.9,
    max_num_neighbors=256,
    p_attr=2,
    p_rep=2,
    r_emb=1
)

In [7]:
cluster_scanner = DBSCANHyperParamScanner(
    n_trials=60,
    n_jobs=6,
    keep_best=30
)

### Put all of the above together in a single Track-Condensation Module

In [8]:
lmodel = TCModule(
    model=model,
    preproc=preproc,
    loss_fct=loss_fct,
    cluster_scanner=cluster_scanner,
    optimizer=partial(torch.optim.Adam, lr=8*1e-4),
    scheduler=partial(torch.optim.lr_scheduler.LinearLR,
                                            start_factor=1,
                                            end_factor=0.1,
                                            total_iters=50)
)

### ...or, start from a checkpoint!

In [4]:
#NOTE: Hardcoded paths!
tc_path = '/home/aj2239/aryaman-gnn-tracking-experiments/notebooks/aryaman-gnn-experiments/mindful-ocelot-of-gaiety/checkpoints/epoch=9-step=77430.ckpt'
lmodel = TCModule.load_from_checkpoint(tc_path)

[05:38:14] DEBUG: Getting class GraphTCNForMLGCPipeline from module gnn_tracking.models.track_condensation_networks
/scratch/gpfs/aj2239/micromamba/envs/gnn/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:198: Attribute 'hc_in' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['hc_in'])`.
[05:38:14] DEBUG: Getting class MLGraphConstruction from module gnn_tracking.models.graph_construction
[05:38:14] DEBUG: Getting class GraphConstructionFCNN from module gnn_tracking.models.graph_construction
[05:38:14] DEBUG: Getting class GraphConstructionHingeEmbeddingLoss from module gnn_tracking.metrics.losses.metric_learning
[05:38:14] DEBUG: Getting class DBSCANHyperParamScanner from module gnn_tracking.postprocessing.dbscanscanner


In [5]:
from pytorch_lightning.loggers import TensorBoardLogger, WandbLogger
from wandb_osh.lightning_hooks import TriggerWandbSyncLightningCallback
from gnn_tracking.utils.nomenclature import random_trial_name

name = random_trial_name()

wandb_logger_oc = WandbLogger(
    project="aryaman-gnn-experiments",
    group="noise-classification-oc",
    offline=True,
    version=name,
    tags=["ground-truth-noise-pixel-oc"],
)

tb_logger_oc = TensorBoardLogger(".", version=name)

────────────────────────── flat-angelfish-of-security ──────────────────────────

In [7]:
trainer_oc = Trainer(
    max_epochs=10,
    accelerator="gpu",
    log_every_n_steps=1,
    callbacks=[
        TriggerWandbSyncLightningCallback(),
        PrintValidationMetrics(),
        ExpandWandbConfig(),
        LearningRateMonitor(logging_interval='epoch', log_momentum=True, log_weight_decay=True),
        EarlyStopping(monitor="total", mode="min", patience=20),
        ModelCheckpoint(save_top_k=2, monitor="total", mode="min"),
    ],
    logger=[
        wandb_logger_oc,
        tb_logger_oc,
    ],
)
trainer_oc.fit(model=lmodel, datamodule=dm)

05:39:31 INFO: This is wandb-osh v1.2.0 using communication directory /home/aj2239/.wandb_osh_command_dir
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100 80GB PCIe') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING `resume` will be ignored since W&B syncing is set to `offline`. Starting a new run with run id flat-angelfish-of-security.


[05:39:33] INFO: DataLoader will load 7743 graphs (out of 7743 available).
[05:39:33] DEBUG: First graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v8/part_1/data21000_s0.pt, last graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v8/part_8/data28999_s0.pt
[05:39:33] INFO: DataLoader will load 4 graphs (out of 1000 available).
[05:39:33] DEBUG: First graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v8/part_9/data29000_s0.pt, last graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v8/part_9/data29003_s0.pt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type                                | Params
-----------------------------------------------------------------
0 | model    | GraphTCNForMLGCPipeline             | 1.9 M 
1 | preproc  | MLGraphConstruction                 | 333 K 
2 | loss_fct | GraphConstructionHingeEmbeddingLoss | 0     
--------------------------------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/scratch/gpfs/aj2239/micromamba/envs/gnn/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.
/scratch/gpfs/aj2239/micromamba/envs/gnn/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.
[05:39:52] WARNING: Could not find config.yaml in log directory


FileNotFoundError: [Errno 2] No such file or directory: 'lightning_logs/flat-angelfish-of-security/config.yaml'